# Install Libaries

In [15]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-community

# Enter Open AI API key 

In [16]:
from getpass import getpass
OPENAPI_KEY=getpass("enter your api key here : ")

enter your api key here :  ········


# Set Environment Variables

In [17]:
import os 
os.environ["OPENAI_API_KEY"]=OPENAPI_KEY

# Load necessary Dependcies and Chatgpt LLM

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# LLM MODELS

In [19]:
chatgpt=ChatOpenAI(model_name="gpt-4",temperature=0.9)

## Access the IT Support queue data

In [20]:
it_support_queue=["Näo consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha. " ,
                    "Ho problemi a stampare i documenti da remoto. II lavoro non viene inviato alla stampante di rete. " ,
                  "我无法远程打印文档。作业未发送至网络打印机。",
                    "fl He Mory BOhTY•1 B CHCTemy yqeTa BpeMeHV1, noRBJIReTcg c0061L1eHb•1e 06 OUJM6Ke. MHe HY*Ha n0MOU1b." ,
                    "Internet badlantlm cok yavas ve bazen tamamen kesiliyor. Yardlm eder misiniz?" ,
                    "He mory YCTaHOBMTb 06HOBneHMe 6e30nacHocTh. r10RBngeTcg KOA OLUV16KH. 110MorhTe, noxanyicra."]
#give the output using JSON output paas

# Define Output Parser

In [27]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel , Field


class itsupportresponse(BaseModel):
    originalmsg:str =Field(Description="This is orginal customer IT support query message ")
    originallang:str =Field(Description="Detected language of the customer e.g. spanish ")
    categorymsg:str =Field(Description="1 to 2 words describing the category of the message ")
    transmsg:str =Field(Description="Translated customer IT support query language in english ")
    response:str =Field(Description="Response to the customer in originallang ")
    transresponse:str =Field(Description="Respnse to the customer in english")

parser=JsonOutputParser(pydantic_object=itsupportresponse)

# create the input prompt for the LLM 

In [29]:
prompt_text = """
Act as Information Technology (IT) customer support agent.
For the support message mentioned below,
use the following output format while generating the output response:
{format_instruction}

Customer IT support message:
{it_support_msg}
"""

from langchain.prompts import PromptTemplate 

prompt = PromptTemplate(
    template=prompt_text,
    input_variables=["it_support_msg"],
    partial_variables={"format_instruction": parser.get_format_instructions()}
)

# create LCEL LLM chain

In [30]:
llm_chain=(prompt|chatgpt|parser)

In [31]:
it_support_queue=["Näo consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha. " ,
                    "Ho problemi a stampare i documenti da remoto. II lavoro non viene inviato alla stampante di rete. " ,
                  "我无法远程打印文档。作业未发送至网络打印机。",
                    "fl He Mory BOhTY•1 B CHCTemy yqeTa BpeMeHV1, noRBJIReTcg c0061L1eHb•1e 06 OUJM6Ke. MHe HY*Ha n0MOU1b." ,
                    "Internet badlantlm cok yavas ve bazen tamamen kesiliyor. Yardlm eder misiniz?" ,
                    "He mory YCTaHOBMTb 06HOBneHMe 6e30nacHocTh. r10RBngeTcg KOA OLUV16KH. 110MorhTe, noxanyicra."]
formatted_msgs=[{"it_support_msg": msg} for msg in it_support_queue ]

In [32]:
responses=llm_chain.map().invoke(formatted_msgs)

In [34]:
print(responses)

[{'originalmsg': 'Näo consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.', 'originallang': 'Portuguese', 'categorymsg': 'Sync Issue', 'transmsg': "I can't sync my contacts with the phone. I always get a failure message.", 'response': 'Sinto muito por estar enfrentando este problema. Você poderia tentar remover a conta e adicioná-la novamente? Isso geralmente resolve problemas de sincronização. Se o problema persistir, entre em contato conosco novamente.', 'transresponse': "I'm sorry you're experiencing this issue. Could you try removing the account and adding it back? This usually solves syncing issues. If the problem persists, please contact us again."}, {'originalmsg': 'Ho problemi a stampare i documenti da remoto. II lavoro non viene inviato alla stampante di rete.', 'originallang': 'Italian', 'categorymsg': 'Printing Issues', 'transmsg': 'I am having trouble printing documents remotely. The job is not being sent to the network printer.', 'respons

In [36]:
import json


data = [{'originalmsg': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.',
         'originallang': 'Portuguese',
         'categoryms': 'Sync issue',
         'transmsg': 'I can’t sync my contacts with the phone. I always get a failure message.',
         'response': 'Sinto muito por estar enfrentando esse problema. Você poderia tentar remover a conta e adicioná-la na nova? Isso geralmente resolve problemas de sincronização. Se o problema persistir, entre em contato conosco novamente.',
         
         'originalmsg': 'Ho problemi a stampare i documenti dal remoto. I lavoro non viene inviata alla stampante del rete.',
         'originallang': 'Italian',
         'categoryms': 'Printing Issue',
         'transmsg': 'I am having trouble printing documents remotely. The job is not being sent to the network printer.',
         'response': 'Assicurati di essere connesso alla stessa rete della stampante. Se il problema persiste, prova a riavviare sia il computer che la stampante del rete.',
         
         'originalmsg': '我无法连接到打印机。作业未被发送至网络打印机。',
         'originallang': 'chinese',
         'categoryms': 'printing issue',
         'transmsg': 'I can’t print the document remotely. The job was not sent to the network printer.',
         'response': '请检查您的网络连接，并确保您的打印机已连接到网络。您可以尝试重新启动您的打印机和计算机。',
         
         'originalmsg': 'Please check your network connection and make sure your printer is connected to the network. You could also try restarting your printer and computer.',
         'originallang': 'english',
         'categoryms': 'Printer Issue',
         'transmsg': 'Please check your network connection and make sure your printer is connected to the network. You could also try restarting your printer and computer.',
         'response': 'If the problem does not resolve, try restarting your printer and computer. If the issue persists, please send a screenshot of the error for further analysis.',
         
         'originalmsg': 'Internet bağlantımla çok yavs ve bazen tamamen kesiliyor. Yardım eder misiniz?',
         'originallang': 'Turkish',
         'categoryms': 'Internet Issue',
         'transmsg': 'My internet connection is very slow and sometimes cuts out completely. Can you help?',
         'response': 'Yes, I can help. Have you tried turning your modem off and on again?',
         
         'originalmsg': 'Мои YТCATBN 60b9neMoC1b. 60nrochenecг BKO OLUV16K. 110MorF., noxanyca.',
         'originallang': 'russian',
         'categoryms': 'Software Issue',
         'transmsg': 'I cannot install the update. The updates keep downloading. Please assist.',
         'response': 'Пожалуйста, проверьте ваше интернет-соединение и убедитесь, что у вас достаточно места на диске для обновления. Если проблема не решается, возможно, потребуется переустановить программное обеспечение.',
         
         'originalmsg': 'Please check your internet connection and make sure you have enough disk space for the update. If the issue doesn\'t resolve, a reinstallation of the software might be necessary.'}]


for item in data:
    for key, value in item.items():
        print(f"{key}: {value}")
    print("-" * 60)  


originalmsg: Please check your internet connection and make sure you have enough disk space for the update. If the issue doesn't resolve, a reinstallation of the software might be necessary.
originallang: russian
categoryms: Software Issue
transmsg: I cannot install the update. The updates keep downloading. Please assist.
response: Пожалуйста, проверьте ваше интернет-соединение и убедитесь, что у вас достаточно места на диске для обновления. Если проблема не решается, возможно, потребуется переустановить программное обеспечение.
------------------------------------------------------------
